# BE 521: Final Project Part 1
Spring 2025

Adapted by Kevin Xie

Updated by Zhongchuan Xu

32 Points

Objective: Predict finger movements from ECoG Recordings

Due: April 10th


# Project Overview

This final project involves predicting finger flexion using intracranial EEG (ECoG) in three human subjects. The data and problem framing come from the 4th BCI Competition (Miller et al. 2008). For the details of the problem, experimental protocol, data, and evaluation, please see the original 4th BCI Competition documentation (included as separate document). The remainder of the current document details your deliverables for part 1 of the project.


## Important Deadlines
* Final Project Part 1 (Canvas)
 * Due: April 10th
 * 32 Points
* Team Registration
 * Due: April 10th
 * 5 Points
* Team Responsibilities (Canvas)
 * Due: April 14th
 * 3 Point
* Checkpoint 1, r > 0.33
 * Due: April 17th
 * 20 Points
* Checkpoint 2, r > 0.45
 * Due: April 24th
 * 15 Points
* End of competition, submit algorithm (Canvas):
 * Due: April 25th
 * 15 Points
* Final Report
 * Due: April 27th
 * 60 Points
* Competition results (Final class session)
 * On: April 30th

The grading is structured so that going the extra mile is definitely rewarded. We want you to show what you've learned this semester, and to have some fun!

## Writing Your Code
To get started with the final project we have provided a a series of method stubs for you to fill out. Your job for part 1 of the final project is to build a prediction pipeline that takes in the ECoG and dataglove finger angle recordings (serving as the data and labels respectively), then uses machine learning methods to generate predicted finger angles from the ECoG signals. The functions you will develop in this assignment are as follows:
* `get_windowed_feats` This function will take in raw ECoG data, and use the 2 following helper functions to filter the data, calculate sliding-window features.
 * `filter_data` This function will apply a filter to the raw data and return cleaned data
 * `get_features` This function will take in a window of cleaned data and return a vector of features for that window
* `create_R_matrix` This function will take in a feature matrix and return a response matrix as an adaptation of the optimal linear decoder method.



## Optimal Linear Decoder
You will use a modified version of the **optimal linear decoder** method as described in Warland et al., 1997. We will recapitulate the method in this section, but consult the paper for more details. Our ultimate goal is to predict the angle of each finger as it moves over time using data recorded from the ECoG channels.

The position data is captured for 300 seconds, which you will split up into $M$ total time bins, and the number of ECoG channels, $\nu$, is 61, 46, and 64 for subject 1, 2, and 3 respectively.

The paradigm we adapt here tries to predict finger angle at a given time window using ECoG features calculated over the preceding $N$ time windows, using
the following steps:

First, $p$ features will be calculated across all $\nu$ ECoG channels over $M$ total time windows to get a feature matrix of shape $\bigl(M, (\nu \times p)\bigr)$

Then, following the approach that Warland et al., 1997 takes, we will construct a row vector corresponding to each time bin, that contains features for all the ECoG channels over the preceding *N* time bins (in the paper, spike counts are their features and they index neurons instead of ECoG channels). Thus, there will be a good amount of redundancy between row vectors of adjacent time bins, but that is okay.

Let $r^{c,\phi}_t$ be the value of the feature in window $t \in \{1,2,\dots,M\}$, channel $c\in\{1,2,\dots,\nu\}$ and with feature $\phi\in\{1,2,\dots,p\}$. Let the response matrix $R \in \mathbb{R}^{M \times (1+N \cdot p \cdot \nu )}$ be defined as:

$$R = \begin{bmatrix}
\mathbf{1} & r^{(1,1)}_1 & r^{(1,1)}_1 & \cdots & r^{(1,1)}_1 & r^{(1,1)}_1 & r^{(1,2)}_1 & \cdots & r^{(1,2)}_1 & \cdots & r^{(\nu,p)}_1 & \cdots & r^{(\nu,p)}_1\\
\mathbf{1} & r^{(1,1)}_1 & r^{(1,1)}_1 & \cdots & r^{(1,1)}_1 & r^{(1,1)}_2 & r^{(1,2)}_1 & \cdots & r^{(1,2)}_2 & \cdots & r^{(\nu,p)}_1 & \cdots & r^{(\nu,p)}_2\\
\mathbf{1} & r^{(1,1)}_1 & r^{(1,1)}_1 & \cdots & r^{(1,1)}_2 & r^{(1,1)}_3 & r^{(1,2)}_1 & \cdots & r^{(1,2)}_3 & \cdots & r^{(\nu,p)}_1 & \cdots & r^{(\nu,p)}_3\\
\vdots   & \vdots     & \vdots     & \ddots & \vdots     & \vdots     & \vdots     & \ddots & \vdots   & \cdots & \vdots         & \ddots & \vdots\\
\mathbf{1} & r^{(1,1)}_1 & r^{(1,1)}_2 & \cdots & r^{(1,1)}_{N-1} & r^{(1,1)}_N & r^{(1,2)}_1 & \cdots & r^{(1,2)}_N & \cdots & r^{(\nu,p)}_1 & \cdots & r^{(\nu,p)}_N\\
\mathbf{1} & r^{(1,1)}_2 & r^{(1,1)}_3 & \cdots &r^{(1,1)}_{N} & r^{(1,1)}_{N+1} & r^{(1,2)}_2 & \cdots & r^{(1,2)}_{N+1} & \cdots & r^{(\nu,p)}_2 & \cdots & r^{(\nu,p)}_{N+1}\\
\vdots   & \vdots     & \vdots     & \ddots & \vdots     & \vdots     & \vdots     & \ddots & \vdots   & \cdots & \vdots         & \ddots & \vdots\\
\mathbf{1} & r^{(1,1)}_{M-N+1} & r^{(1,1)}_{M-N+2} & \cdots & r^{(1,1)}_{M-1} & r^{(1,1)}_M & r^{(1,2)}_{M-N+1} & \cdots & r^{(1,2)}_M & \cdots & r^{(\nu,p)}_{M-N+1} & \cdots & r^{(\nu,p)}_M\\
\end{bmatrix}$$



This is also referred to as the design or feature matrix, with each column being a predictor, or feature. The column of 1’s accounts for the intercept term in linear regression/decoding. Here we are repeating the first windows $N-1$ times as the padding of the first windows. Make sure you understand what this matrix means before moving on.

We denote the target matrix as $Y \in \mathbb{R}^{M \times 5}$ and the prediction matrix (e.g. the predicted finger angles) as $\hat{Y} \in \mathbb{R}^{M \times 5}$. Note that in Warland et al., 1997, this quantity is referred to as the stimulus vector since they are talking about decoding the stimulus from neural data after it. We, on the other hand, are trying to decode finger positions using the ECoG data before it, but we can conveniently use the same method. Our goal is to find some optimal weight matrix or filter $f \in \mathbb{R}^{(1+N \cdot p \cdot \nu ) \times 5}$ that minimizes the mean squared error:

$$f^* = \operatorname{argmin}_{f} \, \mathcal{L}(f) = \operatorname{argmin}_{f} \left\|Y - \hat{Y}\right\|^2,$$
where $\hat{Y} = Rf$

We start with the mean squared error (MSE) loss function:
$$
\mathcal{L}(f) = \left\|Y - R\,f\right\|^2 = (Y - R\,f)^\top (Y - R\,f)
$$

To minimize the loss, we take the derivative with respect to the weight matrix $f$ and set it equal to zero:
$$
\frac{\partial \mathcal{L}}{\partial f} = -2\,R^\top (Y - R\,f) = 0
$$

This implies:
$$
R^\top Y = R^\top R\,f
$$

Assuming that $R^\top R$ is invertible, we solve for $f$:
$$
f = \left(R^\top R\right)^{-1} R^\top Y
$$

This is the the analytic form for the optimal filter $f$ that minimizes the MSE loss.

This equation should take a familiar form. Warland et al., 1997 don’t refer to it as such, but this is exactly the same as linear regression, one of the most commonly used algorithms in practical machine learning. Not only is this algorithm remarkably powerful, but it has a beautiful analytic form for learning the “weights” (here, the $f$ matrix), a rarity in a field where almost all optimizations involve some sort of iterative algorithm. After learning the filter weights $f$, we can calculate the optimal predictions as: $$\hat{Y} = Rf$$

## Dataset
The dataset for part 1 is stored within `final_proj_part1_data.pkl`. The `.pkl` file type is a pickle file, which stores python objects. You can open the `.pkl` file with this code.
```
with open('final_proj_part1_data.pkl', 'rb') as f:
  proj_data = pickle.load(f)
```
This stores the data inside the file as a variable named proj_data.

**NOTE: Python versions don't pickle with each other very well. This pickle file was made in Google Colab. If you are running your own installation of Python and cannot load the file, we recommend you use Colab**

There are 3 subjects, each with their own Data Glove data (the glove they used to capture hand movements), and ECoG data. The data is represented as a dictionary with keys `'data_glove'` and `'ecog'`, storing the data glove and ecog data, respectively. These keys map to python lists of 3 items. Each item is an np.ndarray corresponding to a subject's data. See the pseudcode below.

```
proj_data = {
  'data_glove':[np.ndarray for subject 1, np.ndarray for subject 2, np.ndarray for subject 3],
  'ecog':[np.ndarray for subject 1, np.ndarray for subject 2, np.ndarray for subject 3]
}
```

All np.ndarray shapes for `'data_glove'` should be $(T,5)$, where $T$ is the number of samples in the signal, and 5 is the number of fingers.

The np.ndarray shapes for `'ecog'` are $(T, 61)$, $(T, 46)$, and $(T,64)$, where T is the number of samples in the signal, and each subject had 61, 46, and 64 ecog channels, respectively.

**The sampling rate of the data glove and ecog was 1000 Hz**

<!-- The dataset is also on IEEG
* Subject 1
 * I521_Sub1_Training_ecog - Training ECoG \
 * I521_Sub1_Training_dg - Training Data Glove \
 * I521_Sub1_Leaderboard_ecog - Testing ECoG
* Subject 2
 * I521_Sub2_Training_ecog - Training ECoG \
 * I521_Sub2_Training_dg - Training Data Glove \
 * I521_Sub2_Leaderboard_ecog - Testing ECoG
* Subject 3
 * I521_Sub3_Training_ecog - Training ECoG \
 * I521_Sub3_Training_dg - Training Data Glove \
 * I521_Sub3_Leaderboard_ecog - Testing ECoG -->

Your task is to develop an algorithm to use the ECoG to predict finger movements that are captured by the Data Glove.

# 1. Getting Started (4 pts)
The following sections will walk you through the development of the prediction pipeline.

In [1]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import pearsonr
from scipy import signal as sig

## 1.
Extract the dataglove and ECoG data for each subject from the pickle file. Feel free to copy the code snippet above. Split the data into a training and testing set (at least 50% of the data should be in the training set).

**How many samples are there in the full ECoG recording (before splitting)?** (1 pt)

**How many samples do you have in your training set? In your testing set?** (1 pt)

In [2]:
with open('final_proj_part1_data.pkl', 'rb') as f:
    proj_data = pickle.load(f)

In [4]:
data_glove = proj_data["data_glove"]

In [5]:
data_glove

[array([[-0.11974525, -0.36549759, -0.73717308, -0.51092148, -0.71658897],
        [-0.11974525, -0.36549759, -0.73717308, -0.51092148, -0.71658897],
        [-0.11974525, -0.36549759, -0.73717308, -0.51092148, -0.71658897],
        ...,
        [ 2.0240593 , -0.35267067, -0.19278526, -0.19677258, -0.27182388],
        [ 2.00303459, -0.35165787, -0.19276047, -0.19695473, -0.27204323],
        [ 1.98193359, -0.3506403 , -0.19273472, -0.19713593, -0.27226353]],
       shape=(300000, 5)),
 array([[-0.69935513, -0.12379456, -0.20017529,  4.42933369, -0.51332283],
        [-0.69935513, -0.12379456, -0.20017529,  4.42933369, -0.51332283],
        [-0.69935513, -0.12379456, -0.20017529,  4.42933369, -0.51332283],
        ...,
        [-0.59470844,  0.30090046, -0.35142422,  0.05770493,  0.04432487],
        [-0.59465694,  0.30065536, -0.35142422,  0.0579443 ,  0.04527092],
        [-0.59460258,  0.30041122, -0.35142326,  0.05818367,  0.04621601]],
       shape=(300000, 5)),
 array([[-0.005013

In [6]:
ecog = proj_data["ecog"]

In [10]:
ecog

[array([[  612, -1490,  -996, ...,   205,  1046,  -214],
        [  814, -1367,  -649, ...,   399,  1241,   -71],
        [ 1111, -1182,  -318, ...,   642,  1459,    64],
        ...,
        [   83,  2673,  1550, ...,   790,  1269, -2325],
        [  421,  2650,  1402, ...,   587,  1177, -2254],
        [  823,  2686,  1346, ...,   507,  1155, -2088]],
       shape=(300000, 61), dtype=int32),
 array([[-3900,  -870,  2067, ...,  -229,   639,   185],
        [-3837,  -569,  2067, ...,   -21,   883,   279],
        [-3664,  -323,  2049, ...,   150,  1048,   354],
        ...,
        [  227, -1136,   761, ..., -1567, -2540,   136],
        [  477, -1107,  1059, ..., -1256, -2096,   246],
        [  741, -1094,  1335, ...,  -928, -1680,   344]],
       shape=(300000, 46), dtype=int32),
 array([[ 1888,  -915,   260, ..., -2160,  1388,  -335],
        [ 1219, -1187,  -162, ..., -2841,  1065,  -728],
        [  992, -1030,   -72, ..., -2947,  1136,  -720],
        ...,
        [  227,   436,

There are 300,0000 samples in the full ECoG recording before splitting for each subject. 

In [12]:
train_ecog = []
test_ecog = []
train_glove = []
test_glove = []

train_len = int(0.8*len(ecog[0]))

for subject_idx in range(3):
    ecog_data = proj_data["ecog"][subject_idx]
    glove_data = proj_data["data_glove"][subject_idx]
    train_ecog.append(ecog_data[:train_len])
    test_ecog.append(ecog_data[train_len:])
    train_glove.append(glove_data[:train_len])
    test_glove.append(glove_data[train_len:])

In [14]:
train_len

240000

In [15]:
test_ecog

[array([[ 1222,   575,  -936, ...,  2648,  2208,   944],
        [ 1318,   469,  -855, ...,  2645,  2193,  1389],
        [ 1510,   600,  -469, ...,  2729,  2323,  1866],
        ...,
        [   83,  2673,  1550, ...,   790,  1269, -2325],
        [  421,  2650,  1402, ...,   587,  1177, -2254],
        [  823,  2686,  1346, ...,   507,  1155, -2088]],
       shape=(60000, 61), dtype=int32),
 array([[ -839,  2907,  2295, ...,   959,   136,   587],
        [ -818,  2866,  2203, ...,  1082,   486,   735],
        [ -772,  2883,  2024, ...,  1251,   803,   898],
        ...,
        [  227, -1136,   761, ..., -1567, -2540,   136],
        [  477, -1107,  1059, ..., -1256, -2096,   246],
        [  741, -1094,  1335, ...,  -928, -1680,   344]],
       shape=(60000, 46), dtype=int32),
 array([[4401, 1578, 3111, ..., 1472, 3515, 1590],
        [4349, 1411, 2866, ..., 1362, 3325, 1268],
        [3966,  951, 2411, ...,  912, 2820,  745],
        ...,
        [ 227,  436,  -61, ...,  566, 1894

There are 240,000 samples in my training set and 60,000 samples in my testing set after I split them up.

## 2.
Next, complete the `filter_data` function. Test it using the raw data extracted in the prior step. What filter types and cutoff frequencies did you use? (2 pts)

In [16]:
def filter_data(raw_eeg, fs=1000):
  """
  Write a filter function to clean underlying data.
  Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
  Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
  distorting the underlying data!

  Input:
    raw_eeg (samples x channels): the raw signal
    fs: the sampling rate (1000 for this dataset)
  Output:
    clean_data (samples x channels): the filtered signal
  """
  # Define the filter parameters
  lowcut = 1.0  # Low cutoff frequency in Hz
  highcut = 50.0  # High cutoff frequency in Hz
  order = 4  # Filter order
  nyquist_freq = 0.5 * fs  # Nyquist frequency
  low = lowcut / nyquist_freq  # Low cutoff frequency in normalized units
  high = highcut / nyquist_freq  # High cutoff frequency in normalized units
  
  # Design the filter
  b, a = sig.butter(order, [low, high], btype='bandpass')

In [ ]:
#Your code here

Your answer here

# 2. Calculating Features (12 points)

Here you will complete the `get_windowed_feats` and `get_features` functions.

## 1.
We will calculate features across sliding time windows. if we use a suggested window length of 100ms with a 50ms window overlap, how many feature windows, $M$, will we have if we computed features using all the data in a given subject? Feel free to re-use code from previous homeworks.

In [ ]:
#Your code here

Your answer here

## 2.
Now complete the `get_features` function. Please create **4 or more** different features to calculate for each channel in each time window. Features may include the average time-domain voltage, or the average frequency-domain magnitude in consecutive 15Hz frequency bands, bandpower of relevant frequency bands, etc...

In [ ]:
def get_features(filtered_window, fs=1000):
  """
    Write a function that calculates features for a given filtered window.
    Feel free to use features you have seen before in this class, features that
    have been used in the literature, or design your own!

    Input:
      filtered_window (window_samples x channels): the window of the filtered ecog signal
      fs: sampling rate
    Output:
      features (channels x num_features): the features calculated on each channel for the window
  """
  pass

## 3.
Now finish the `get_windowed_feats` function by putting the `filter_data` and `get_features` functions together to return a feature vector for each time window.

In [ ]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
  """
    Write a function which processes data through the steps of filtering and
    feature calculation and returns features. Points will be awarded for completing
    each step appropriately (note that if one of the functions you call within this script
    returns a bad output, you won't be double penalized). Note that you will need
    to run the filter_data and get_features functions within this function.

    Inputs:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)
      window_length: the window's length
      window_overlap: the window's overlap
    Output:
      all_feats (num_windows x (channels x features)): the features for each channel for each time window
        note that this is a 2D array.
  """
  pass

# 3. Creating the Response Matrix (6 points)
In this section, you will develop code for your *create_R_matrix* function

## 1.
For our set of 62 channels in subject 1, what would the dimensions of the $R$ matrix be if we calculated 6 different feature types per channel, and $N = 3$ time bins where the number of total time bins $M$ is the number you calculated in 2.1? (1pt)

In [ ]:
#Your code here

Your answer here

## 2.
We do not have feature data to fill out the first $N-1$ data rows in the R matrix that will be used to predict the first $N-1$ finger angles. One way to work around this is to append a copy of the first rows $N-1$ times of your feature matrix to the beginning of your feature matrix before calculating R. Make this adjustment in `create_R_matrix`, then compute the response matrix R. You can test whether your function is running correctly by running `create_R_matrix` with data from `testRfunction.pkl` using 3 windows and verifying that the quantity `np.mean(R)` is 25.4669 (5 points).

In [ ]:
def create_R_matrix(features, N_wind):
  """
  Write a function to calculate the R matrix

  Input:
    features (num_windows x (channels x features)):
      the features you calculated using get_windowed_feats
    N_wind: number of windows to use in the R matrix

  Output:
    R (samples x (N_wind*channels*features))
  """
  pass

In [ ]:
#Your code here

Your answer here

# 4. ML Training and Testing (10 points)
Here we will use the optimal linear decoder framework to predict finger angles, and additionally you will use one or more models of your own choosing to make the prediction.

NOTE: This is a regression task - we are asking you to predict the movement of each finger for each subject.

## 1.

Calculate the linear filter $f$ as defined above for all 5 finger angles using features calculated from your training data. You will have to first down-sample the finger flexion data so that your feature matrix, $R$, and your flexion data have the same number of time windows.

You will likely find [np.matmul](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) or operator @, and [np.linalg.inv](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) to be useful.

In [ ]:
#Your code here

Your answer here

## 2.
Try one other machine learning models using your features and finger angle labels. Look back through previous homeworks to get some ideas

In [ ]:
#your code here

Your answer here

## 3.
Produce predictions on the testing set for each finger angle.

Report your correlations here using the linear filter, and when using the other model(s) that you tried, as follows:

> For each subject, calculate the correlation coefficient between the predicted and test finger angles for each finger separately.

> You therefore should have 15 correlations: five per subject, with three subjects.

You will find  [pearsonr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html) to be helpful and already imported.

In [ ]:
#Your code here

Your answer here